In [1]:
# # 형태소 분석기 Mecab 설치
# !git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
# %cd Mecab-ko-for-Google-Colab
# !bash install_mecab-ko_on_colab190912.sh

In [2]:
# Mecab 다시 설치 (with konlpy) thanks teddy...!
# https://teddylee777.github.io/colab/colab-mecab
!curl -s https://raw.githubusercontent.com/teddylee777/machine-learning/master/99-Misc/01-Colab/mecab-colab.sh | bash



--2022-06-24 03:46:41--  https://www.dropbox.com/s/9xls0tgtf3edgns/mecab-0.996-ko-0.9.2.tar.gz?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.65.18, 2620:100:6021:18::a27d:4112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.65.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/dl/9xls0tgtf3edgns/mecab-0.996-ko-0.9.2.tar.gz [following]
--2022-06-24 03:46:42--  https://www.dropbox.com/s/dl/9xls0tgtf3edgns/mecab-0.996-ko-0.9.2.tar.gz
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc08cf52988db8bc0b6b67c3a9f2.dl.dropboxusercontent.com/cd/0/get/BnxU8dtvqIISYYjaMUsxSAZXfN8MxVcSf7gWnTfIHw-iNNEVPjcqZvE4H6deVdUnnXB8yvndl9V3a8MV9_aouf7uCJ19H1vpptO_0CNtgFnd9R1mnKeKwG4H6X9jq-xnBMOTO7w_zDbi17mWzc5RmieRpxot1qHTDRs5iY_kOYJmOW2gq7zmhmOCuHR9a24OkjY/file?dl=1# [following]
--2022-06-24 03:46:42--  https://uc08cf52988db8bc0b6b67c3a9f2.dl.dropboxusercontent.com/cd/0/g

# Soynlp part

In [5]:
# # 설치가 느려서 lovit의 형태소 분석기 soynlp를 사용하겠음.
# !pip install soynlp

In [6]:
# from soynlp.tokenizer import RegexTokenizer

# 훈련 데이터 테스트 데이터

In [7]:
import urllib.request
import pandas as pd

In [8]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt", filename="ratings_train.txt")
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt", filename="ratings_test.txt")

('ratings_test.txt', <http.client.HTTPMessage at 0x7fc9569cbf50>)

In [9]:
train_df = pd.read_table('ratings_train.txt')
test_df = pd.read_table('ratings_test.txt')

In [10]:
train_df.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [11]:
print(f"훈련 데이터 개수: {len(train_df)}")
print(f"테스트 데이터 개수: {len(test_df)}")

훈련 데이터 개수: 150000
테스트 데이터 개수: 50000


In [12]:
# # soynlp tokenizing test
# t = RegexTokenizer()
# t.tokenize(train_df.document[0])

In [13]:
# Mecab 잘 설치되었나 확인
from konlpy.tag import Mecab

m = Mecab()
m.morphs(train_df.document[0])

['아', '더', '빙', '.', '.', '진짜', '짜증', '나', '네요', '목소리']

# 필드 정의하기(torchtext.data)

In [14]:
# 원활한 실습을 위해 이전버전 torchtext를 사용함
!pip install torchtext==0.6

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 64 kB 2.5 MB/s 
     |████████████████████████████████| 1.2 MB 18.1 MB/s 
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.12.0
    Uninstalling torchtext-0.12.0:
      Successfully uninstalled torchtext-0.12.0


In [15]:
from torchtext import data
# AttributeError: module 'torchtext.data' has no attribute 'Field' OTZ...진챠로..
# Mecab 설치가 안돼요... -> 됨

In [16]:
# # 아까 테스트했던 lovit의 정규식기반 토크나이저를 울며 겨자먹기로 사용
# tokenizer = RegexTokenizer()

In [17]:
# Mecab 을 tokenizer로 이용하겠음
tokenizer = Mecab()

In [20]:
ID = data.Field(sequential=False,
                use_vocab=False)

TEXT = data.Field(sequential=True, 
                  use_vocab=True,
                  tokenize=tokenizer.morphs, 
                  lower=True, 
                  batch_first=True, 
                  fix_length=20)

LABEL = data.Field(sequential=False, 
                   use_vocab=False, 
                   is_target=True)

# 데이터 셋 만들기

In [21]:
from torchtext.data import TabularDataset

In [22]:
train_data, test_data = TabularDataset.splits(
    path='.', 
    train='ratings_train.txt', 
    test='ratings_test.txt', 
    format='tsv', 
    fields=[('id', ID), ('text', TEXT), ('label', LABEL)], 
    skip_header=True)

In [23]:
# data 확인
len(train_data), len(test_data)

(150000, 50000)

In [24]:
print(vars(train_data[2]))
# -> soynlp  regex 특성상 한국어가 띄어쓰기 없이 있으면 토큰화 진행안됨

{'id': '10265843', 'text': ['너무', '재', '밓었다그래서보는것을추천한다'], 'label': '0'}


# 단어 집합 만들기(Vocabulary)

정의한 필드에 `build_vocab()`을 사용하면 단어 집합을 생성함.

In [25]:
TEXT.build_vocab(train_data, min_freq=10, max_size=10000)

In [26]:
len(TEXT.vocab)
# 단어 집합크기는 UNK, PAD를 포함해 총 10002개

10002

In [27]:
TEXT.vocab.stoi

defaultdict(<bound method Vocab._default_unk_index of <torchtext.vocab.Vocab object at 0x7fc888ac88d0>>,
            {'<unk>': 0,
             '<pad>': 1,
             '.': 2,
             '이': 3,
             '는': 4,
             '영화': 5,
             '다': 6,
             '고': 7,
             '하': 8,
             '도': 9,
             '의': 10,
             '가': 11,
             '은': 12,
             '에': 13,
             '을': 14,
             '보': 15,
             '한': 16,
             '..': 17,
             '게': 18,
             ',': 19,
             '들': 20,
             '!': 21,
             '지': 22,
             '를': 23,
             '있': 24,
             '없': 25,
             '?': 26,
             '좋': 27,
             '나': 28,
             '었': 29,
             '만': 30,
             '는데': 31,
             '너무': 32,
             '봤': 33,
             '적': 34,
             '안': 35,
             '정말': 36,
             '로': 37,
             '음': 38,
             '으로': 39,
           

# 부록 : batch_first = True of False?

- batch_first : 미니 배치 차원을 맨 앞으로 하여 데이터를 불러올 것인지 여부. (False가 기본값)

이게 무슨 말인 지 모르겠음

In [28]:
import torch

# batch_first=True
bf_T = torch.tensor([[  31,  191,   24,  133,  445,  115,   42,   10,  149,    2, 3581, 6601,
            0,   12,  172,   74,  358,  806,    6,  425],
        [   9,   98,   12,   10,   20,    7,  157, 2520,  285,   11, 1384,   46,
          921, 4255,   16,   10,    0,  702,   82,    5],
        [   9,  323,  148,   10,   25,   17,  110, 3109,   80,   44,  291, 4427,
            3,  778, 3286,   17,    0,    2, 1308,  193],
        [  10,    7,   49, 8950,   18,  189,  184,    5,    2, 1890,   17,   10,
            0,  118,   24,   62,  141,    2,  162,   16],
        [   9,  574, 4312, 1147,   64, 2621,    3,  283,  499,   16,   21,  138,
            0,    5,    0, 5994,    2, 1462,   12,    2]])

print(bf_T)
print(bf_T.size())

tensor([[  31,  191,   24,  133,  445,  115,   42,   10,  149,    2, 3581, 6601,
            0,   12,  172,   74,  358,  806,    6,  425],
        [   9,   98,   12,   10,   20,    7,  157, 2520,  285,   11, 1384,   46,
          921, 4255,   16,   10,    0,  702,   82,    5],
        [   9,  323,  148,   10,   25,   17,  110, 3109,   80,   44,  291, 4427,
            3,  778, 3286,   17,    0,    2, 1308,  193],
        [  10,    7,   49, 8950,   18,  189,  184,    5,    2, 1890,   17,   10,
            0,  118,   24,   62,  141,    2,  162,   16],
        [   9,  574, 4312, 1147,   64, 2621,    3,  283,  499,   16,   21,  138,
            0,    5,    0, 5994,    2, 1462,   12,    2]])
torch.Size([5, 20])


배치 크기가 5이기 때문에 5개의 샘플이 20길이로 추출됨

In [29]:
# batch_first=False 인 경우
bf_F = torch.tensor([[  31,    9,    9,   10,    9],
        [ 191,   98,  323,    7,  574],
        [  24,   12,  148,   49, 4312],
        [ 133,   10,   10, 8950, 1147],
        [ 445,   20,   25,   18,   64],
        [ 115,    7,   17,  189, 2621],
        [  42,  157,  110,  184,    3],
        [  10, 2520, 3109,    5,  283],
        [ 149,  285,   80,    2,  499],
        [   2,   11,   44, 1890,   16],
        [3581, 1384,  291,   17,   21],
        [6601,   46, 4427,   10,  138],
        [   0,  921,    3,    0,    0],
        [  12, 4255,  778,  118,    5],
        [ 172,   16, 3286,   24,    0],
        [  74,   10,   17,   62, 5994],
        [ 358,    0,    0,  141,    2],
        [ 806,  702,    2,    2, 1462],
        [   6,   82, 1308,  162,   12],
        [ 425,    5,  193,   16,    2]])

print(bf_F)
print(bf_F.size())

tensor([[  31,    9,    9,   10,    9],
        [ 191,   98,  323,    7,  574],
        [  24,   12,  148,   49, 4312],
        [ 133,   10,   10, 8950, 1147],
        [ 445,   20,   25,   18,   64],
        [ 115,    7,   17,  189, 2621],
        [  42,  157,  110,  184,    3],
        [  10, 2520, 3109,    5,  283],
        [ 149,  285,   80,    2,  499],
        [   2,   11,   44, 1890,   16],
        [3581, 1384,  291,   17,   21],
        [6601,   46, 4427,   10,  138],
        [   0,  921,    3,    0,    0],
        [  12, 4255,  778,  118,    5],
        [ 172,   16, 3286,   24,    0],
        [  74,   10,   17,   62, 5994],
        [ 358,    0,    0,  141,    2],
        [ 806,  702,    2,    2, 1462],
        [   6,   82, 1308,  162,   12],
        [ 425,    5,  193,   16,    2]])
torch.Size([20, 5])


이건 어쩔티비? 샘플을 계속 5배치로 가져가는 시스템??

NLP의 경우 sequential 한 데이터이기 때문에 [batch_size, len, dim] 형태로 가져가는 것이 맞다!

-> 여기서 batch_first는 첫번째에 batch를 넣을것인지 조절하는 것임!
